In [1]:
import pandas as pd
import numpy as np
import sklearn as scikit_learn
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import urllib.request
# import mecab
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os

C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_price=pd.read_excel('../data/가격데이터_증감비교_라벨링.xlsx')

In [6]:
df_price.head()

,도축일,경락단가,경락단가_평균,label
0,2018-01-07,3691.039572,3691.039572,0
1,2018-01-14,3743.336317,3743.336317,1
2,2018-01-21,3743.359986,3743.359986,1
3,2018-01-28,3835.679915,3835.679915,1
4,2018-02-04,4276.198330,4276.198330,1


In [13]:
df_price['가격등락비율']=0
for i in range(len(df_price)-1):
    if i==0:
        df_price['가격등락비율'][i]==0
    else:
        df_price['가격등락비율'][i]=((df_price['경락단가'][i]-df_price['경락단가'][i-1])/df_price['경락단가'][i])*100

C:\Users\jaemin\AppData\Local\Temp\ipykernel_22996\1833849704.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_price['가격등락비율'][i]=((df_price['경락단가'][i]-df_price['경락단가'][i-1])/df_price['경락단가'][i])*100


In [15]:
 df_price.to_excel('../data/가격데이터_증감비교_라벨링_가격ㄷ.xlsx')

,도축일,경락단가,경락단가_평균,label,가격등락비율
0,2018-01-07,3691.039572,3691.039572,0,0.000000
1,2018-01-14,3743.336317,3743.336317,1,1.397062
2,2018-01-21,3743.359986,3743.359986,1,0.000632
3,2018-01-28,3835.679915,3835.679915,1,2.406873
4,2018-02-04,4276.198330,4276.198330,1,10.301637
5,2018-02-11,4724.707319,4724.707319,1,9.492842
6,2018-02-18,4103.632481,4103.632481,2,-15.134758
7,2018-02-25,3794.280663,3794.280663,2,-8.153108
8,2018-03-04,4094.180790,4094.180790,1,7.325034
9,2018-03-11,3868.237571,3868.237571,2,-5.840986
